
# Image Classification with CNN


Welcome to second assignment of Deep learning lesson. Follow the instructions and complete the assignment.

------------

## Task

**Build an image classifier with Convolutional Neural Networks for the Fashion MNIST dataset. This data set includes 10 labels of different clothing types with 28 by 28 *grayscale* images. There is a training set of 60,000 images and 10,000 test images.**

    Label	Description
    0	    T-shirt/top
    1	    Trouser
    2	    Pullover
    3	    Dress
    4	    Coat
    5	    Sandal
    6	    Shirt
    7	    Sneaker
    8	    Bag
    9	    Ankle boot
    
 

In [ ]:
classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

## The Data

**Download the fashion mnist dataset using Keras.**

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [ ]:
np.set_printoptions(suppress=True) # 10 üssü e' li gosterimleri kapatmak icin 

In [ ]:
print(tf.__version__)
## Bunu bizden sonra bu notebooka bakacak olan kişi benim hangi tensorflow versiyonuyla çalıştığımı bilsin diye yazıyorum.

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

## Visualizing the Data

**Using matplotlib show an any image from the data set.**

In [ ]:
print("There are ", len(X_train), "images in the training dataset")     
print("There are ", len(X_test), "images in the test dataset")  

In [ ]:
# Checking the shape of one image
X_train[5].shape

Rastgele bir resim secip sekline baktik. Bu datasetindeki resimler 28x28 oldugu icin resim kaliteleri cok kötü. 28x28 tek channel' li resimler var(Renkli degil):

28x28x3 olsaydi resim renkli demek olurdu (3 farkli renk kanali var demektir.)28x28' lik bir array elde ettik. Buradaki sayilar renk kodlari. 0 sayisi siyahi temsil ediyor; pixeller 255' e dogru gittikce renk beyazlasiyor :

28x28' lik bir array elde ettik. Buradaki sayilar renk kodlari. 0 sayisi siyahi temsil ediyor; pixeller 255' e dogru gittikce renk beyazlasiyor :

In [ ]:
X_train[5]

Yukaridaki 9. ornegi __imshow__ ile gorsellestirdik ve gercekte ne olduguna baktik.

In [ ]:
plt.imshow(X_train[9])
# plt.matshow(X_train[5])

Targettaki y degerlerine baktik. Bu degerler aslinda class' lari temsil ediyor ama modele bu haliyle verirsek modelimiz bu reel degerler ile regression modeli yapilacagini düşünür. Bu yüzden encoder uygulayarak degerleri kategorik hale cevirmemiz gerekecek :

In [ ]:
y_train

In [ ]:
num_rows, num_cols = 2, 5       # Datadan 2 tane row, 5 tane column sectik.
f, ax = plt.subplots(2, 5, figsize=(12,6),
                     gridspec_kw={'wspace':0.03, 'hspace':0.02}, 
                     squeeze=True)

for r in range(num_rows):
    for c in range(num_cols):
      
        image_index = r * 5 + c
        ax[r,c].axis("off")
        ax[r,c].imshow( X_train[image_index], cmap='gray')
        ax[r,c].set_title('No. %d' % y_train[image_index])
plt.show()
plt.close()

## Preprocessing the Data

**Normalize the X train and X test using max value of the image arrays.**

In [ ]:
X_train[9].min()

In [ ]:
X_train[9].max()

Goruntu islemede tum pixeller 0 ile 255 arasinda deger aldigi icin normalize etmek cok kolay olur. Resimdeki her bir pixel degeri 255' e bölünürse tüm pixel degerleri 0 ile 1 arasinda deger alir. asagidaki islemle train setlerdeki her bir deger 255' e bölünmüs oldu :

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
X_train[9].min()

Normalize isleminden sonra min deger 0, max deger 1 oldu.

In [ ]:
X_train[9].max()

Normalization islemi yapildiktan sonra resmin yapisi bozulmaz, yine ayni resmi elde edebiliriz :

In [ ]:
plt.imshow(X_train[9])

**Reshape the X into 4 dimension**

__28, 28, 1 --->__  28x28' lik tek channel (gray scale). Ayni resmi 3 boyutlu olarak ifade ediyoruz. Bu islemi tüm X degerleri icin yaptik (y degerleri icin yapilmaz) :

In [ ]:
## Yaptığımız birşey yok aynı resmi 3 boyutlu olarak ifade ediyoruz.

X_train = X_train.reshape(X_train.shape[0],28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train

**Convert the y_train and y_test values to be one-hot encoded for categorical analysis.**

y degerlerini categoric hale getirmek icin keras' tan to_categorical' i import ettik. Bu sekilde model artik regression degil, classification yapmasi gerektigini bilecek :

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train ve y_test' i to_categorical icine verdik ve 10 tane class oldugunu yanina belirttik. (Bunu yazmak zorunda degiliz, kendisi anlar fakat yine de belirtmek iyi olur, Good Practice). Ciktida 0 ve 9 dahil 10 tane row oldu :

In [ ]:
Y_train = to_categorical(y_train, 10) 
Y_test = to_categorical(y_test, 10)
Y_test.shape

Label    Description
0        T-shirt/top
1        Trouser
2        Pullover
3        Dress
4        Coat
5        Sandal
6        Shirt
7        Sneaker
8        Bag
9        Ankle boot

Mesela 0. indexteki resim 5 imis. Bu durumda diger degerlerin hepsi 0 olup 5' e karsilik gelen deger 1 olmus :

In [ ]:
Y_train[0].reshape(-1,1)

Y' de 10 tane column var dolayisiyla classification yaptigimiz icin output layer'da 10 tane nöron olacak. (Kac class var ise o kadar nöron vardir)

## Building the Model

**Create the model**

* 2D Convolutional Layer, filters=28 and kernel_size=(3,3)
* Pooling Layer where pool_size = (2,2) strides=(1,1)

* Flatten Layer
* Dense Layer (128 Neurons, but feel free to play around with this value), RELU activation

* Final Dense Layer of 10 Neurons with a softmax activation

**Then compile the model with these parameters: loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']**

Sequential ile layer' lari arka arkaya dizecegiz. Bununla birlikte kullanacagimiz her seyi import ettik :

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [ ]:
model = Sequential()

model.add(Conv2D(filters=28, kernel_size=(3,3),input_shape=(28, 28, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2, 2))) ## Max pooling convalition(filtre) işleminden sonra yapılır.

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.summary()

###  Model Training 

**Train the model with arbitrary amount of epochs.**

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=25, validation_data=(X_test,Y_test))

In [ ]:
model.metrics_names

In [ ]:
summary = pd.DataFrame(model.history.history)
summary.head()

**Plot values of metrics you used in your model.**

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

**Train the model using earlystop.**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_test,Y_test),callbacks=[early_stop])

In [ ]:
summary = pd.DataFrame(model.history.history)
summary.head()

**Plot values of metrics you used in your model.**

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

### Model Evaluation

**Show the accuracy,precision,recall,f1-score the model achieved on the x_test data set.**

In [ ]:
print(model.evaluate(X_test, Y_test, verbose=1))

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix, plot_confusion_matrix

In [ ]:
preds = model.predict(X_test)

In [ ]:
preds[0]

In [ ]:
predictions = np.argmax(preds, axis=1)

In [ ]:
predictions

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
print(confusion_matrix(y_test, predictions))

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Prediction

In [ ]:
my_image = X_test[56]

In [ ]:
plt.imshow(my_image)

In [ ]:
image_prediction = model.predict(my_image.reshape(1,28,28,1))

In [ ]:
image_prediction

In [ ]:
result=np.argmax(image_prediction, axis=1)

In [ ]:
result

In [ ]:
classes[int(result)]

# End of Assignment